In [ ]:
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

llm = ChatGroq(
    model_name="mixtral-8x7b-32768",
    temperature=0.7
)

prompt = ChatPromptTemplate.from_messages([
    ("system", 
        """
        Producer Function Names:
            AddProduct()
            FindListedProduct()
            UpdateListedProduct()
        
        Consumer Function Names:
            FindAllProducts(category: string)
                
        You are a chatbot with limited capabilities. There are two types of users with unique functions for each user. Functions with 
        empty parameters dont take any arguments and functions do not take any other parameters than specified above. You can only return 
        the function which is most suitable for the given scenario. You are currently adressing the user type Consumer. From the given 
        user input map the user's query into the appropriate given function and format it into JSON with this structure:
            {{
                name : functionName(queryParams)
            }}
        """
    ),
    ("user", "{input}")
])

parser = JsonOutputParser(pydantic_object={
    "type": "object",
    "properties": {
        "name": {"type": "string"}
    }
})

chain = prompt | llm | parser

def parse_product(description: str) -> dict:
    result = chain.invoke({"input": description})
    print(json.dumps(result, indent=2))

        
description = """Find all vegetables in Erode area"""

parse_product(description)

{
  "name": "FindAllProducts(category: vegetables)"
}
